<a href="https://colab.research.google.com/github/sherkhalil/COVID19/blob/master/BERT_Classifier_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
dataset = '/content/drive/My Drive/NLP/twitter_sentiments.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd 
data = pd.read_csv(dataset, encoding='ISO-8859-1',names=["sentiment", "COL2", "COL3", "COL4","COL5","contents"])
data.tail()

,sentiment,COL2,COL3,COL4,COL5,contents
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
data = data.rename(columns={"sentiment": "Label", "contents": "text"})

In [ ]:
data.tail()

,Label,COL2,COL3,COL4,COL5,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
data.Label.value_counts()

4    800000
0    800000
Name: Label, dtype: int64

In [ ]:
data['Label_Numeric'] = data['Label'].map({4:1, 0:0})

In [ ]:
data.Label_Numeric.value_counts()

1    800000
0    800000
Name: Label_Numeric, dtype: int64

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 3.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.5-cp36-none-any.whl size=30317 sha256=9b43fea5b1e03fd2dec8aac5a7b87ccf7da9b9b607ef329b9f3ced82ba0d0039
  Stored in directory: /root/.cache/pip/wheels/2e/70/a2/be357037dd2cbdcaeb0add1fdf083be6a600ca65ee1f68751c
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=1007194ac4d07b1eda2f22e925023b2771b325f3169e2614a64f5a3ba9d60f31
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=2fa0313595a1e0e4ae6cc08a27b7d0c49a7406ed367bf18c70d9089c2f4af877
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 7.4MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
data.shape

(1600000, 7)

In [ ]:
data = data.sample(20000)

In [ ]:
data['text']

1283553                       happy cause my test is finish 
54863                           needs to diet and exercise. 
459473     I'm like Father MacKenzie in that Beatles' son...
334828           Porn, please stop following me on twitter. 
1043311             brittany sooo wasn't allowed to do that 
                                 ...                        
101579                     theres nothing to do im so bored 
792857                        I want to watch Transformers. 
581995     @gailbrand How are you Gail? Still suffering f...
1444863                 @BeauGiles 1 API request per launch 
388734     Arrgghhh... Kenapa BB gua crash lagi ya?? Twic...
Name: text, Length: 20000, dtype: object

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
reviews = []
sentences = list(data['text'])
for sen in sentences:
    reviews.append(preprocess_text(sen))

In [ ]:
print(data.columns.values)

['Label' 'COL2' 'COL3' 'COL4' 'COL5' 'text' 'Label_Numeric']


In [ ]:
data.Label_Numeric.unique()

array([1, 0])

In [ ]:
import numpy as np
y = data['Label_Numeric']

y = np.array(list(map(lambda x: 1 if x==1 else 0, y)))

In [ ]:
y

array([1, 0, 0, ..., 0, 1, 0])

In [ ]:
reviews[10]

' JackAllTimeLow dude that sweet '

In [ ]:
y[10]

1

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [ ]:
tokenized_reviews[0]

[3407, 3426, 2026, 3231, 2003, 3926]

In [ ]:
reviews_with_len = [[review, y[i], len(review)]  for i, review in enumerate(tokenized_reviews)]

In [ ]:
import random
random.shuffle(reviews_with_len)

In [ ]:
reviews_with_len.sort(key=lambda x: x[2])

In [ ]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 2), dtype=int32, numpy=
 array([[ 4931,     0],
        [ 3403,     0],
        [ 2748,     0],
        [14409,     0],
        [ 9364,     0],
        [19453,     0],
        [ 4771,     0],
        [22708,     0],
        [19453,     0],
        [ 7697,     0],
        [ 2498,     0],
        [ 6928,     0],
        [22708,     0],
        [14978,     0],
        [ 2103,     0],
        [ 6928,     0],
        [14409,     0],
        [19453,     0],
        [ 5082,     0],
        [ 3752,     0],
        [10651,     0],
        [19142,     0],
        [ 3017,     0],
        [ 9344,     0],
        [14978,     0],
        [ 7464,     0],
        [ 5011,  2135],
        [27555, 24459],
        [ 2204,  2851],
        [ 2036,  2420],
        [ 3260,  8885],
        [ 4399,  2154]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 0], dtype=int

In [ ]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="BERT_Classifier_A"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
563/563 [==============================] - 50s 89ms/step - loss: 0.5529 - accuracy: 0.7181
Epoch 2/5
563/563 [==============================] - 51s 91ms/step - loss: 0.3262 - accuracy: 0.8623
Epoch 3/5
563/563 [==============================] - 49s 86ms/step - loss: 0.0890 - accuracy: 0.9680
Epoch 4/5
563/563 [==============================] - 48s 86ms/step - loss: 0.0581 - accuracy: 0.9790
Epoch 5/5
563/563 [==============================] - 49s 87ms/step - loss: 0.0386 - accuracy: 0.9863


In [ ]:
text_model.summary()


Model: "BERT_Classifier_A"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6104400   
_________________________________________________________________
conv1d (Conv1D)              multiple                  40100     
_________________________________________________________________
conv1d_1 (Conv1D)            multiple                  60100     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  80100     
_________________________________________________________________
global_max_pooling1d (Global multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  77056     
_________________________________________________________________
dropout (Dropout)            multiple            

In [ ]:
results = text_model.evaluate(test_data)
print(results)